In [1]:
url ="https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/523261/monthly-bulletin-16-04.rtf"

In [2]:
from bs4 import BeautifulSoup

# For now just do all the rtf links
url = "https://www.gov.uk/government/statistics/prison-population-figures-2015"

import requests
r = requests.get(url)
soup = BeautifulSoup(r.content,"lxml")

In [3]:
s1 = set([(a.text, a["href"]) for a in soup.findAll("a") if ".rtf" in a["href"] and a.text != ""])

In [4]:
url = "https://www.gov.uk/government/statistics/prison-population-figures-2016"

import requests
r = requests.get(url)
soup = BeautifulSoup(r.content,"lxml")
s2 = set([(a.text, a["href"]) for a in soup.findAll("a") if ".rtf" in a["href"] and a.text != ""])

In [5]:
a.text

u'\xa9 Crown copyright'

In [6]:
all_links = s1.union(s2)
all_links

{(u'Population bulletin: monthly April 2015',
  '/government/uploads/system/uploads/attachment_data/file/426306/prison-popualtion-april-2015.rtf'),
 (u'Population bulletin: monthly April 2016',
  '/government/uploads/system/uploads/attachment_data/file/523261/monthly-bulletin-16-04.rtf'),
 (u'Population bulletin: monthly August 2015',
  '/government/uploads/system/uploads/attachment_data/file/460246/prison-population-august-2015.rtf'),
 (u'Population bulletin: monthly December 2015',
  '/government/uploads/system/uploads/attachment_data/file/490685/prison-population-figures-december-2015.rtf'),
 (u'Population bulletin: monthly February 2015',
  '/government/uploads/system/uploads/attachment_data/file/412827/prison-population-figures-february-2015.rtf'),
 (u'Population bulletin: monthly February 2016',
  '/government/uploads/system/uploads/attachment_data/file/507108/prison-population-february-2016.rtf'),
 (u'Population bulletin: monthly January 2015',
  '/government/uploads/system/uplo

In [7]:
from pyth.plugins.rtf15.reader import Rtf15Reader
from pyth.plugins.xhtml.writer import XHTMLWriter
import pandas as pd
import urllib2
from cStringIO import StringIO

import re

In [8]:
def get_table(doc):
    final_data = []

    header = False
    for c in doc.content:
        full_p = c.content.__repr__().lower()
        if "capacity" in full_p and "use cna" in full_p:

            header = True
            continue

        if header:
            row= re.split(r"\t", c.content[0].content[0])
            if len(row) == 7 :
                final_data.append(row)
    df = pd.DataFrame(final_data, columns = ["prison_name","baseline_cna", "in_use_cna", "operational_capacity", "population", "perc_pop_to_used_cna", "perc_acc_available"])
    df.iloc[:,1:] = df.iloc[:,1:].replace("%", "", regex=True).replace(",", "", regex=True)

    for c in df.columns:
        df[c]= pd.to_numeric(df[c], errors='ignore')
    cols = [c for c in df.columns if "perc" in c]
    df.loc[:,cols] = df.loc[:,cols]/100
    return df

In [9]:
dfs = []
for link in all_links:
    text = link[0]
    link = link[1]
    rtf_file = urllib2.urlopen("http://www.gov.uk" +link)
    rtf_file = StringIO(rtf_file.read())
    doc = Rtf15Reader.read(rtf_file)
    df = get_table(doc)
    df["month_text"] = text
    dfs.append(df)

In [10]:
final = pd.concat(dfs)

In [11]:
final["month_text"] = final["month_text"].replace("Population bulletin: monthly", "", regex=True)

In [12]:
from dateutil import parser
final["month"] = final["month_text"].apply(parser.parse)

In [14]:
final.to_csv("all_pop_data.csv", index=False, encoding="utf-8")